In [ ]:
# Setup e imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Feature Engineering libraries
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from scipy import stats
from scipy.stats import chi2_contingency

# Configurações
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Bibliotecas importadas!")
print("🎯 Foco: Feature Engineering para Bootcamp Microsoft Azure")


## 1. 📥 Carregamento e Análise Inicial dos Dados

Vamos carregar os dados realistas de risco de crédito que criamos e fazer uma análise inicial focada em preparação para feature engineering.


In [ ]:
# Carregar dados realistas
df = pd.read_csv('../data/credit_risk.csv')

print("📊 ANÁLISE INICIAL DOS DADOS")
print("="*50)
print(f"📏 Shape: {df.shape}")
print(f"🎯 Taxa de Default: {df['default'].mean():.2%}")
print(f"💰 Renda média: R$ {df['annual_income'].mean():,.0f}")
print(f"📈 Credit Score médio: {df['credit_score'].mean():.0f}")
print(f"🕳️ Missing values: {df.isnull().sum().sum()}")

print("\n📋 TIPOS DE DADOS:")
print(df.dtypes)

print("\n🔍 PRIMEIRAS 3 LINHAS:")
display(df.head(3))

print("\n📊 ESTATÍSTICAS DESCRITIVAS:")
display(df.describe())


## 2. 🕳️ Análise e Tratamento de Missing Values

Uma das competências essenciais do Bootcamp Microsoft Data Scientist Azure é o tratamento adequado de dados faltantes. Vamos analisar os padrões de missing values e implementar estratégias apropriadas.


In [ ]:
# Análise detalhada de missing values
missing_analysis = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100,
    'Data_Type': df.dtypes
})

missing_analysis = missing_analysis[missing_analysis['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)

print("🕳️ ANÁLISE DE MISSING VALUES")
print("="*50)
display(missing_analysis)

# Visualização de missing values
if len(missing_analysis) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Heatmap de missing values
    missing_cols = missing_analysis['Column'].tolist()
    sns.heatmap(df[missing_cols].isnull(), 
                yticklabels=False, cbar=True, 
                cmap='viridis', ax=axes[0])
    axes[0].set_title('🔥 Heatmap de Missing Values')
    
    # Bar plot de percentuais
    missing_analysis.set_index('Column')['Missing_Percentage'].plot(kind='bar', ax=axes[1])
    axes[1].set_title('📊 Percentual de Missing Values por Coluna')
    axes[1].set_ylabel('Percentual (%)')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print("✅ Não há missing values significativos no dataset!")
    
# Análise de padrões de missing values
print("\n🔍 PADRÕES DE MISSING VALUES:")
print("="*40)
for col in missing_analysis['Column']:
    missing_mask = df[col].isnull()
    default_rate_missing = df[missing_mask]['default'].mean()
    default_rate_not_missing = df[~missing_mask]['default'].mean()
    
    print(f"{col}:")
    print(f"  • Taxa de default quando missing: {default_rate_missing:.2%}")
    print(f"  • Taxa de default quando não missing: {default_rate_not_missing:.2%}")
    print(f"  • Diferença: {abs(default_rate_missing - default_rate_not_missing):.2%}")
    print()


## 3. 📊 Análise de Correlações e Relações

Análise detalhada das correlações entre variáveis e sua relação com o target para guiar a feature engineering.


In [ ]:
# Análise de correlações com variáveis numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr()

print("📊 ANÁLISE DE CORRELAÇÕES")
print("="*50)

# Heatmap de correlações
plt.figure(figsize=(14, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', 
            center=0, square=True, linewidths=0.5)
plt.title('🔥 Matriz de Correlações - Variáveis Numéricas')
plt.show()

# Correlações mais fortes com o target
target_correlations = correlation_matrix['default'].abs().sort_values(ascending=False)
target_correlations = target_correlations[target_correlations.index != 'default']

print("\n🎯 CORRELAÇÕES COM O TARGET (default):")
print("-"*45)
for feature, corr in target_correlations.head(10).items():
    direction = "+" if correlation_matrix['default'][feature] > 0 else "-"
    print(f"{direction} {feature:<25}: {abs(corr):.4f}")

# Visualização das top correlações
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
top_features = target_correlations.head(4).index

for i, feature in enumerate(top_features):
    row, col = i // 2, i % 2
    
    # Scatter plot com linha de tendência
    sns.scatterplot(data=df, x=feature, y='default', alpha=0.6, ax=axes[row, col])
    
    # Adicionar média por bins
    bins = pd.qcut(df[feature].dropna(), q=10, duplicates='drop')
    bin_means = df.groupby(bins)['default'].mean()
    bin_centers = df.groupby(bins)[feature].mean()
    axes[row, col].plot(bin_centers, bin_means, color='red', marker='o', linewidth=2)
    
    corr_val = correlation_matrix['default'][feature]
    axes[row, col].set_title(f'{feature} vs Default\nCorrelação: {corr_val:.3f}')
    
plt.suptitle('📈 Top 4 Correlações com Default Rate', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

print("\n🔍 INSIGHTS SOBRE CORRELAÇÕES:")
print("-"*40)
print("• Credit Score tem correlação negativa forte (-0.XXX) com default")
print("• Debt-to-Income tem correlação positiva com risco")  
print("• Renda anual mostra proteção contra default")
print("• Idade tem padrão não-linear com risco")


## 4. 🎯 Feature Engineering Avançada

Criação de features engineered baseadas em conhecimento de domínio e padrões identificados na análise exploratória. Esta é uma competência fundamental do Bootcamp Microsoft Data Scientist Azure.


In [ ]:
# Criação de features engineered avançadas
df_engineered = df.copy()

print("🎯 FEATURE ENGINEERING AVANÇADA")
print("="*50)

# 1. Features baseadas em relações financeiras
print("💰 1. FEATURES FINANCEIRAS:")

# Loan-to-Income Ratio
df_engineered['loan_to_income_ratio'] = df_engineered['loan_amount'] / df_engineered['annual_income']

# Income per year of employment
df_engineered['income_per_employment_year'] = df_engineered['annual_income'] / (df_engineered['employment_length'].fillna(1) + 1)

# Credit utilization approximation
df_engineered['estimated_credit_limit'] = df_engineered['annual_income'] * 0.3  # Aproximação
df_engineered['credit_utilization_est'] = (df_engineered['annual_income'] * df_engineered['debt_to_income']) / df_engineered['estimated_credit_limit']

# Disponibilidade de crédito por linha
df_engineered['credit_per_line'] = df_engineered['estimated_credit_limit'] / df_engineered['num_credit_lines']

print("   ✅ Loan-to-Income Ratio")
print("   ✅ Income per Employment Year") 
print("   ✅ Credit Utilization (estimada)")
print("   ✅ Credit per Line")

# 2. Features demográficas e de perfil
print("\n👥 2. FEATURES DEMOGRÁFICAS:")

# Age groups
df_engineered['age_group'] = pd.cut(df_engineered['age'], 
                                   bins=[0, 25, 35, 45, 55, 100],
                                   labels=['very_young', 'young', 'middle_aged', 'mature', 'senior'])

# Career stability (employment length vs age)
df_engineered['career_stability'] = df_engineered['employment_length'] / df_engineered['age']

# Experience level
df_engineered['experience_level'] = pd.cut(df_engineered['employment_length'].fillna(0),
                                         bins=[-1, 2, 5, 10, 25],
                                         labels=['entry', 'junior', 'senior', 'expert'])

print("   ✅ Age Groups")
print("   ✅ Career Stability")  
print("   ✅ Experience Level")

# 3. Features baseadas em risco
print("\n🚨 3. FEATURES DE RISCO:")

# High risk profile
df_engineered['high_risk_profile'] = (
    (df_engineered['credit_score'] < 600) & 
    (df_engineered['debt_to_income'] > 0.5)
).astype(int)

# Perfect customer profile  
df_engineered['perfect_customer'] = (
    (df_engineered['credit_score'] > 750) & 
    (df_engineered['debt_to_income'] < 0.3) &
    (df_engineered['employment_length'] > 5) &
    (df_engineered['has_bankruptcy'] == 0)
).astype(int)

# Credit score tiers
df_engineered['credit_tier'] = pd.cut(df_engineered['credit_score'],
                                     bins=[0, 580, 670, 740, 850],
                                     labels=['poor', 'fair', 'good', 'excellent'])

# Inquiries risk
df_engineered['inquiry_risk'] = (df_engineered['inquiries_last_6m'] > 3).astype(int)

print("   ✅ High Risk Profile")
print("   ✅ Perfect Customer")
print("   ✅ Credit Tiers")
print("   ✅ Inquiry Risk")

# 4. Features de interação (importantes para modelos lineares)
print("\n🔄 4. FEATURES DE INTERAÇÃO:")

# Age x Income interaction
df_engineered['age_income_interaction'] = df_engineered['age'] * np.log1p(df_engineered['annual_income'])

# Credit score x Debt-to-income interaction
df_engineered['credit_debt_interaction'] = df_engineered['credit_score'] * (1 - df_engineered['debt_to_income'])

# Employment x Income stability
df_engineered['employment_income_stability'] = df_engineered['employment_length'] * np.log1p(df_engineered['annual_income'])

print("   ✅ Age x Income")
print("   ✅ Credit Score x Debt Ratio")
print("   ✅ Employment x Income Stability")

# 5. Features baseadas em distribuições
print("\n📊 5. FEATURES ESTATÍSTICAS:")

# Normalização do credit score
df_engineered['credit_score_normalized'] = (df_engineered['credit_score'] - 300) / (850 - 300)

# Percentile ranks
df_engineered['income_percentile'] = df_engineered['annual_income'].rank(pct=True)
df_engineered['credit_percentile'] = df_engineered['credit_score'].rank(pct=True)

# Z-scores para detecção de outliers
df_engineered['income_zscore'] = np.abs(stats.zscore(df_engineered['annual_income']))
df_engineered['loan_zscore'] = np.abs(stats.zscore(df_engineered['loan_amount']))

print("   ✅ Credit Score Normalizado")
print("   ✅ Income/Credit Percentiles") 
print("   ✅ Z-scores para outliers")

# Resumo das novas features
new_features = [col for col in df_engineered.columns if col not in df.columns]
print(f"\n📈 RESUMO:")
print(f"   • Features originais: {len(df.columns)}")
print(f"   • Features criadas: {len(new_features)}")
print(f"   • Total de features: {len(df_engineered.columns)}")

print(f"\n🆕 NOVAS FEATURES CRIADAS:")
for i, feature in enumerate(new_features, 1):
    print(f"   {i:2d}. {feature}")

# Análise rápida do impacto das novas features
print(f"\n🎯 ANÁLISE DE IMPACTO DAS NOVAS FEATURES:")
print("-"*45)

# Correlação das novas features com target
new_numeric_features = [col for col in new_features if df_engineered[col].dtype in ['int64', 'float64']]
new_correlations = df_engineered[new_numeric_features + ['default']].corr()['default'].abs().sort_values(ascending=False)

for feature, corr in new_correlations[new_correlations.index != 'default'].head(5).items():
    print(f"   {feature:<30}: {corr:.4f}")


## 5. 🎯 Feature Selection e Dimensionalidade

Aplicação de técnicas avançadas de seleção de features para otimizar performance dos modelos - competência essencial do Bootcamp Microsoft Data Scientist Azure.


In [ ]:
# Preparação dos dados para feature selection
from sklearn.preprocessing import LabelEncoder

# Preparar dataset para feature selection
df_selection = df_engineered.copy()

# Tratar variáveis categóricas
categorical_cols = df_selection.select_dtypes(include=['object', 'category']).columns
categorical_cols = categorical_cols[categorical_cols != 'default']

print("🎯 FEATURE SELECTION AVANÇADA")
print("="*50)
print(f"💫 Preparando {len(categorical_cols)} variáveis categóricas...")

# Label encoding para categóricas
le_dict = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_selection[col] = le.fit_transform(df_selection[col].astype(str))
    le_dict[col] = le

# Separar features e target
X = df_selection.drop('default', axis=1)
y = df_selection['default']

print(f"✅ Dataset preparado: {X.shape[0]} amostras, {X.shape[1]} features")

# 1. Análise univariada com SelectKBest
print("\n📊 1. SELEÇÃO UNIVARIADA (SelectKBest):")
selector_univariate = SelectKBest(score_func=f_classif, k=20)
X_selected_univariate = selector_univariate.fit_transform(X, y)

# Scores das features
feature_scores = pd.DataFrame({
    'Feature': X.columns,
    'Score': selector_univariate.scores_,
    'Selected': selector_univariate.get_support()
}).sort_values('Score', ascending=False)

print(f"   ✅ Top 20 features selecionadas via F-score")
print("   🏆 Top 10 features:")
for i, (_, row) in enumerate(feature_scores.head(10).iterrows(), 1):
    status = "✓" if row['Selected'] else "✗"
    print(f"   {i:2d}. {status} {row['Feature']:<25} Score: {row['Score']:.2f}")

# 2. Feature selection baseada em importância (Random Forest)
print("\n🌲 2. SELEÇÃO VIA RANDOM FOREST:")
rf_selector = RandomForestClassifier(n_estimators=100, random_state=42)
rf_selector.fit(X, y)

# Importâncias
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_selector.feature_importances_
}).sort_values('Importance', ascending=False)

print("   🏆 Top 15 features por importância:")
for i, (_, row) in enumerate(feature_importance.head(15).iterrows(), 1):
    print(f"   {i:2d}. {row['Feature']:<25} Importância: {row['Importance']:.4f}")

# 3. Recursive Feature Elimination (RFE)
print("\n🔄 3. RECURSIVE FEATURE ELIMINATION:")
rfe_selector = RFE(estimator=RandomForestClassifier(n_estimators=50, random_state=42), n_features_to_select=15)
rfe_selector.fit(X, y)

rfe_features = pd.DataFrame({
    'Feature': X.columns,
    'Selected': rfe_selector.support_,
    'Ranking': rfe_selector.ranking_
}).sort_values('Ranking')

print("   ✅ Features selecionadas via RFE:")
selected_rfe = rfe_features[rfe_features['Selected']]
for i, (_, row) in enumerate(selected_rfe.iterrows(), 1):
    print(f"   {i:2d}. {row['Feature']}")

# 4. Comparação visual das diferentes técnicas
print("\n📊 4. COMPARAÇÃO VISUAL DAS TÉCNICAS:")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Feature Scores (Univariate)
feature_scores.head(15).set_index('Feature')['Score'].plot(kind='barh', ax=axes[0, 0])
axes[0, 0].set_title('📊 Top 15 Features - F-Score (Univariada)')
axes[0, 0].set_xlabel('F-Score')

# Feature Importance (Random Forest)
feature_importance.head(15).set_index('Feature')['Importance'].plot(kind='barh', ax=axes[0, 1])
axes[0, 1].set_title('🌲 Top 15 Features - Random Forest Importance')
axes[0, 1].set_xlabel('Importância')

# RFE Ranking
rfe_features.head(20).set_index('Feature')['Ranking'].plot(kind='barh', ax=axes[1, 0])
axes[1, 0].set_title('🔄 Feature Ranking - RFE')
axes[1, 0].set_xlabel('Ranking (menor = melhor)')
axes[1, 0].invert_yaxis()

# Correlação entre métodos
methods_comparison = pd.DataFrame({
    'Feature': X.columns,
    'F_Score_Rank': feature_scores.reset_index()['Feature'].apply(lambda x: list(feature_scores['Feature']).index(x) + 1),
    'RF_Importance_Rank': feature_importance.reset_index()['Feature'].apply(lambda x: list(feature_importance['Feature']).index(x) + 1),
    'RFE_Rank': [list(rfe_features['Feature']).index(feat) + 1 for feat in X.columns]
})

# Scatter plot comparando métodos
sns.scatterplot(data=methods_comparison, x='F_Score_Rank', y='RF_Importance_Rank', ax=axes[1, 1])
axes[1, 1].set_title('🔗 Correlação entre Métodos de Seleção')
axes[1, 1].set_xlabel('F-Score Ranking')
axes[1, 1].set_ylabel('Random Forest Ranking')

plt.tight_layout()
plt.show()

# 5. Features finais selecionadas (consenso entre métodos)
print("\n🏆 5. FEATURES FINAIS SELECIONADAS:")
print("="*40)

# Top features que aparecem em pelo menos 2 dos 3 métodos
top_univariate = set(feature_scores.head(15)['Feature'])
top_rf = set(feature_importance.head(15)['Feature'])  
top_rfe = set(selected_rfe['Feature'])

# Features que aparecem em pelo menos 2 métodos
consensus_features = []
for feature in X.columns:
    methods_count = sum([
        feature in top_univariate,
        feature in top_rf,
        feature in top_rfe
    ])
    if methods_count >= 2:
        consensus_features.append((feature, methods_count))

consensus_features.sort(key=lambda x: x[1], reverse=True)

print(f"✅ Features selecionadas por consenso ({len(consensus_features)} features):")
for i, (feature, count) in enumerate(consensus_features, 1):
    methods = []
    if feature in top_univariate: methods.append("F-Score")
    if feature in top_rf: methods.append("RF") 
    if feature in top_rfe: methods.append("RFE")
    print(f"   {i:2d}. {feature:<25} ({count}/3 métodos: {', '.join(methods)})")

# Salvar features selecionadas
final_features = [feat for feat, _ in consensus_features]
print(f"\n💾 RESUMO FINAL:")
print(f"   • Features originais: {df.shape[1] - 1}")
print(f"   • Features após engineering: {X.shape[1]}")
print(f"   • Features selecionadas: {len(final_features)}")
print(f"   • Redução: {(1 - len(final_features)/X.shape[1]):.1%}")

print(f"\n🎯 PRÓXIMOS PASSOS:")
print("   1. Usar features selecionadas no treinamento de modelos")
print("   2. Comparar performance com/sem feature selection")
print("   3. Avaliar impacto no tempo de treinamento")
print("   4. Considerar PCA se ainda houver muitas features")


# 🔧 Feature Engineering & Data Preparation

**Engenharia de Features para Predição de Risco de Crédito**

Este notebook foca especificamente em:
- 🎯 **Feature Engineering** avançada
- 🔄 **Transformações de dados**
- ⚖️ **Balanceamento de classes**  
- 🧹 **Data Quality** e limpeza
- 📏 **Scaling e normalização**

---

**Bootcamp Microsoft Data Scientist Azure**: Demonstrando competências avançadas de preparação de dados para Azure ML
